In [ ]:
#datasets have been used
# ap_2010.csv - Data on AP test results
# class_size.csv - Data on class size
# demographics.csv - Data on demographics
# graduation.csv - Data on graduation outcomes
# hs_directory.csv - A directory of high schools
# sat_results.csv - Data on SAT scores
# survey_all.txt - Data on surveys from all schools
# survey_d75.txt - Data on surveys from New York City district 75

#reading datasets into a dictionary
import pandas as pd
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]
data = {}
data['ap_2010'] = pd.read_csv('schools/ap_2010.csv')
data['class_size'] = pd.read_csv('schools/class_size.csv')
data['demographics'] = pd.read_csv('schools/demographics.csv')
data['graduation'] = pd.read_csv('schools/graduation.csv')
data['hs_directory'] = pd.read_csv('schools/hs_directory.csv')
data['sat_results'] = pd.read_csv('schools/sat_results.csv')


In [ ]:
#exploring  dataset dictionary
data['sat_results'].head(5)
data['sat_results'].info()

for k in data.keys():
    print(data[k].head(5))

In [ ]:
#read in txt files into dataframes and combine them together
all_survey = pd.read_csv('schools/survey_all.txt', delimiter = '\t', encoding = 'windows-1252')
d75_survey = pd.read_csv('schools/survey_d75.txt', delimiter = '\t', encoding = 'windows-1252')
survey = pd.concat([all_survey,d75_survey] , axis = 0)
survey.head(5)

In [ ]:
#remiving unnecessary columns from the dataset and adding to dataset dict
survey['DBN'] = survey['dbn'] #changing to uppercase to match the other dataframes
survey = survey.loc[:,["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11",
                       "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11",
                       "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11",
                       "com_tot_11", "eng_tot_11", "aca_tot_11"]]

data['survey'] = survey
data['survey'].info()

In [ ]:
#unifying DBN code in "Class_size" and "hs_history"
hs_directory = data['hs_directory']
hs_directory['DBN'] = hs_directory['dbn']
class_size = data['class_size']

#forming the DBN code from CSD and SCHOOL CODE
def change_csd(csd):
    csd = str(csd)
    if len(csd) < 2:
        csd = csd.zfill(2)
    
    return csd   
class_size['padded_csd'] = class_size['CSD'].apply(change_csd)
data['hs_directory'] = hs_directory
class_size['DBN'] = class_size['padded_csd'] + class_size['SCHOOL CODE']
data['class_size'] = class_size
data['class_size'].head(5)

In [ ]:
#combining results for Math Reading and Writing into a SAT result
sat_result = data['sat_results']
sat_result['SAT Math Avg. Score'] = pd.to_numeric(sat_result['SAT Math Avg. Score'],errors = 'coerce')
sat_result['SAT Critical Reading Avg. Score'] = pd.to_numeric(sat_result['SAT Critical Reading Avg. Score'],errors = 'coerce')
sat_result['SAT Writing Avg. Score'] = pd.to_numeric(sat_result['SAT Writing Avg. Score'],errors = 'coerce')
sat_result['sat_score'] = sat_result['SAT Math Avg. Score'] + sat_result['SAT Critical Reading Avg. Score']+ sat_result['SAT Writing Avg. Score']
data['sat_result'] = sat_result
data['sat_result'].head(5)

In [ ]:
#extracting latitude and longtitude info for each school
import re
def coords_lat(str):
    lat = re.findall("\(.+\)",str)
    lat = lat[0].split(',')[0].replace('(','')
    return lat

hs_directory = data['hs_directory']
hs_directory['lat'] = hs_directory['Location 1'].apply(coords_lat)

data['hs_directory'] = hs_directory
def coords_lon(str):
    lon = re.findall("\(.+\)",str)
    lon = lon[0].split(',')[1].replace(')','')
    return lon

hs_directory['lon'] = hs_directory['Location 1'].apply(coords_lon)
hs_directory['lat'] = pd.to_numeric(hs_directory['lat'], errors='coerce')
hs_directory['lon'] = pd.to_numeric(hs_directory['lon'], errors='coerce')
data['hs_directory'] = hs_directory
data['hs_directory'].head(5)